In [36]:
import numpy as np
import pandas as pd

In [37]:
import os

In [38]:
df = pd.read_csv('/content/ElectricityDeptDataset.csv')

In [39]:
df

,AT,V,AP,RH,PE
0,8.34,40.77,1010.84,90.01,480.48
1,23.64,58.49,1011.40,74.20,445.75
2,29.74,56.90,1007.15,41.91,438.76
3,19.07,49.69,1007.22,76.79,453.09
4,11.80,40.66,1017.13,97.20,464.43
...,...,...,...,...,...
9563,15.12,48.92,1011.80,72.93,462.59
9564,33.41,77.95,1010.30,59.72,432.90
9565,15.99,43.34,1014.20,78.66,465.96
9566,17.65,59.87,1018.58,94.65,450.93


In [40]:
df.rename(columns={'AT': 'Average Temperature', 'V': 'Exhaust Vacuum','AP': 'Ambient Pressure',
                   'RH': 'Relative Humidity ','PE': 'Net Hourly Electrical Energy Output'}, inplace=True)
df.head()

,Average Temperature,Exhaust Vacuum,Ambient Pressure,Relative Humidity,Net Hourly Electrical Energy Output
0,8.34,40.77,1010.84,90.01,480.48
1,23.64,58.49,1011.40,74.20,445.75
2,29.74,56.90,1007.15,41.91,438.76
3,19.07,49.69,1007.22,76.79,453.09
4,11.80,40.66,1017.13,97.20,464.43


In [41]:
df.corr()["Net Hourly Electrical Energy Output"].sort_values(ascending=False)

Net Hourly Electrical Energy Output    1.000000
Ambient Pressure                       0.518429
Relative Humidity                      0.389794
Exhaust Vacuum                        -0.869780
Average Temperature                   -0.948128
Name: Net Hourly Electrical Energy Output, dtype: float64

In [42]:
#Preparing and training the model

x = df.drop('Net Hourly Electrical Energy Output',axis = 1).values
y = df['Net Hourly Electrical Energy Output'].values
print(x)
print(y)

[[   8.34   40.77 1010.84   90.01]
 [  23.64   58.49 1011.4    74.2 ]
 [  29.74   56.9  1007.15   41.91]
 ...
 [  15.99   43.34 1014.2    78.66]
 [  17.65   59.87 1018.58   94.65]
 [  23.68   51.3  1011.86   71.24]]
[480.48 445.75 438.76 ... 465.96 450.93 451.67]


In [43]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.25,random_state=42)
print(xtrain.shape)
print(xtest.shape)
print(ytrain.shape)
print(ytest.shape)


(7176, 4)
(2392, 4)
(7176,)
(2392,)


In [44]:
# Building and training neural network models

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

In [45]:
# Iniialising ANN and adding layers

ann = Sequential()
ann.add(Dense(units=6, activation="relu"))  #Adding First Hidden Layer
ann.add(Dense(units=6, activation="relu"))  # Adding Second Hidden Layer
ann.add(Dense(units=1))   # Adding Output Layer

In [46]:
# compiling and training the ANN
ann.compile(optimizer='Adam', loss='mean_squared_error')

In [47]:
ann.fit(x=xtrain,y=ytrain, epochs=100, batch_size=32,validation_data=(xtest,ytest), callbacks=EarlyStopping(monitor='val_loss',patience=4))

Epoch 1/100
225/225 [==============================] - 1s 3ms/step - loss: 488309.2188 - val_loss: 70399.8828
Epoch 2/100
225/225 [==============================] - 1s 2ms/step - loss: 11893.2861 - val_loss: 202.5567
Epoch 3/100
225/225 [==============================] - 0s 2ms/step - loss: 174.0235 - val_loss: 157.3102
Epoch 4/100
225/225 [==============================] - 1s 2ms/step - loss: 159.0704 - val_loss: 152.8169
Epoch 5/100
225/225 [==============================] - 1s 3ms/step - loss: 156.0423 - val_loss: 151.1582
Epoch 6/100
225/225 [==============================] - 1s 3ms/step - loss: 154.2324 - val_loss: 149.0334
Epoch 7/100
225/225 [==============================] - 1s 3ms/step - loss: 152.1202 - val_loss: 146.8822
Epoch 8/100
225/225 [==============================] - 1s 3ms/step - loss: 149.8492 - val_loss: 144.5418
Epoch 9/100
225/225 [==============================] - 1s 2ms/step - loss: 147.4134 - val_loss: 141.9004
Epoch 10/100
225/225 [==========================

In [49]:
# Evaluating the model and making predictions

ann.evaluate(xtrain, ytrain)

225/225 [==============================] - 0s 1ms/step - loss: 29.1832


29.183244705200195

In [50]:
ann.evaluate(xtest, ytest)

75/75 [==============================] - 0s 1ms/step - loss: 27.0302


27.030235290527344

In [51]:
predictions = ann.predict(xtest)
predictions_df = pd.DataFrame(np.ravel(predictions),columns=['Predictions'])
comparison_df = pd.concat([pd.DataFrame(ytest,columns=['Real values']), predictions_df],axis=1)
comparison_df

75/75 [==============================] - 0s 1ms/step


,Real values,Predictions
0,433.27,434.290405
1,438.16,434.809967
2,458.42,465.119415
3,480.82,478.350067
4,441.41,437.322296
...,...,...
2387,446.70,451.706665
2388,430.82,425.970062
2389,440.10,435.413574
2390,439.39,435.862457


As we can see in abouve comparison dataframe, our model's predictions pretty well closer to actual value.

In [52]:
# Let's evaluate the model performance

from sklearn import metrics
print("MAE:",metrics.mean_absolute_error(ytest,predictions))
print ("MSE:",metrics.mean_squared_error(ytest,predictions))
print("RMSE:",np.sqrt(metrics.mean_squared_error(ytest,predictions)))

MAE: 4.099064737275293
MSE: 27.03023186733069
RMSE: 5.199060671633934


The Mean Absolute Error is just 3.6 which is so small which shows that the model is almost as the actual values.

In [53]:
# R^2
metrics.explained_variance_score(ytest,predictions)

0.9132570186452051

This shows our model predict 93% of the target correctly.